In [1]:
import dspy
import anaconda_ai.integrations.litellm  # noqa: F401

In [2]:
lm = dspy.LM(
    "anaconda/openhermes-2.5-mistral-7b/q4_k_m",
    cache=False,
    optional_params={"load_params": {"ctx_size": 512}},
)
dspy.configure(lm=lm)

In [3]:
fact_checking = dspy.asyncify(dspy.ChainOfThought("claims -> verdicts: list[bool]"))

res = await fact_checking(
    claims=["Python was released in 1991.", "Python is a compiled language."]
)
res

✓ OpenHermes-2.5-Mistral-7B_Q4_K_M.gguf (running)

Prediction(
    reasoning="Verdict 1: The claim states that Python was released in 1991. This is incorrect, as Python was first released in 1991. However, it's not clear if the claim means the first release of Python or its creation.\n\nVerdict 2: The claim states that Python is a compiled language. This is correct. Python code is typically compiled into bytecode before being executed by an interpreter.",
    verdicts=[True, False]
)